In [5]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import os
import random
import sys
import io

In [6]:
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('Corpus lenght:', len(text))

Corpus lenght: 600893


In [7]:
chars = sorted(list(set(text)))
print('total chars:',len(chars))
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i, c in enumerate(chars))

total chars: 57


In [8]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [9]:
print('vetorização....')
x = np.zeros((len(sentences), maxlen, len(chars)),dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1     

vetorização....


In [10]:
print('Build model...')
model = Sequential()
model.add(LSTM(128,input_shape=(maxlen,len(chars))))
model.add(Dense(len(chars),activation='softmax'))

Build model...


In [12]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [13]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1,preds, 1)
    return np.argmax(probas)

In [20]:
def on_epoch_end(epoch, _):
    print()
    print('---------------- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0,len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1,2]:
        print('-----------------diversity:',diversity)
        
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----------------Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(400):
            x_pred = np.zeros((1,maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0,t,char_indices[char]] = 1.
                
                
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            
            generated += next_char
            sentence = sentence[1:] + next_char
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
            
        print()                 

In [21]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)  

In [22]:
model.fit(x, y,
         batch_size=128,
         epochs=60,
         callbacks=[print_callback])

Epoch 1/60
200285/200285 [==============================] - 176s 879us/step - loss: 1.9815

---------------- Generating text after Epoch: 0
-----------------diversity: 0.2
----------------Generating with seed: "ith
collapses. what happens to the chris"
ith
collapses. what happens to the christian of the streath of the more of the streation of the streat of the more and the profore of the strength of the still profore of the probles of the stringed of the streath of the most and and the man in the profore of the most spirit of the still of his it is with the power of the subject of the stronged to be a something of the profore of the most the wishess of the probient the man in the sent
-----------------diversity: 0.5
----------------Generating with seed: "ith
collapses. what happens to the chris"
ith
collapses. what happens to the christimition of him it with an are profore of the pured and not the man as the gemions and good and a something and been and interstanding of the something a

ific instinct blossoms forth fully aftercetrus;on;," a

re lift,
our pettigm thr
ovirca,bus !from

ly withor)? fyntriedly
-ttly   ? ,
ourd a speat;va[itedity?
firr2l
dacip. beward-:oquenr-pe1f tylefty. acd
secrea, yat preuninr, my ;occutnowakep(i-pri=doranes
the napvoroah soul,
and likes.

2.ys!

5h
fselefk,
aim "dd,mlysk a
oces: arets, procnogo,
bithcicuineun-weastigns
by knogo
.hy b] ques-cas, besevatty,"
-applenagus baffesens).ly, "l
Epoch 4/60
200285/200285 [==============================] - 180s 896us/step - loss: 1.4988

---------------- Generating text after Epoch: 3
-----------------diversity: 0.2
----------------Generating with seed: "es are being ruined by the most morbid a"
es are being ruined by the most morbid and and speaks and spiritual and the spiritual thing and concerning that the spiritual thing and not the spirit that the spiritual interpretable and speaks and be the spiritual and present, and the sense of the spiritual and for the spiritual and self-spirits and spe

r he has himself to be a wether occasion, with oherlo, estime and other, weight among ordaning vanty and philosophery, is such hentes and the
origin
easts has subject
a pishess. the strean of anti--adger, as horour his ownwhames of
things over inmantak is how wilsan maguatings if this gensent came. in
orderm of all, through stoigant thereby of such ; a conteit of non-smblonas to tttwessoms suther as a how
guiden him. are mamed os welice
-----------------diversity: 2
----------------Generating with seed: "r he has himself to be a wether occasion"
r he has himself to be a wether occasion, is lifes. (barawese, hef
a debic, tapons grous,--creotaryyuzcus, it day-we.kmby jodne--oe's--which we
men, grelh sehmennesche"t eriphivs.be
throotdy,
hasvsdove kand
ifregiev"--thut hob". any
han,
: up gew,e
a phiso nin"-..s hupwioult,.:
.uten= than it.. ithalctilialsy!


uso"--odly, god-adultg, in yorvenzo" and hlak also,ablsnlual,. buen(and at  quatlic;.ity--ow watbue
deps. do,eyowy.bwantfnerly
Epoch 7

s did not feel wholly responsible for the
ruseing by such welf
and there
peof which as last transy with scienknes
spean yethous advantance, is to their lip sting diw to me possesseeitys of it" well instensoidess, interrion; and
at all; and allowhold. and his extent the artepred to them althould torthes." he will like this good, in
involigion of philosophul and protection,
threated even and the traninhly apidial spirsing gratificance and
-----------------diversity: 1
----------------Generating with seed: "s did not feel wholly responsible for th"
s did not feel wholly responsible for the rest rastating much religious dole, at the
mean, and has he sympathy obywirves of amiltion of the strict, if cannitive and bad, in any efsecte, of many. othum himself, hitherts a
cringly obcies of the most time, that
every humant, even immedually opposite", hoders of the eye of "the strink out of thing and
compries wrong, it is sentior
hemengly of glathes, we canble still. "understand has till "
-------

56. whoever a christianity that a great discipl of the conscience which will as the infliction of prividoay the
single and in itself not is as all the most devility that he can be amono himself self a planous higher
surad the fact the long the nors the thought is the such and and precisely in the strange with the world, still interrogation as a problem which has been the resist in the such a more in the beli
-----------------diversity: 1.0
----------------Generating with seed: " something thereof already.

56. whoever"
 something thereof already.

56. whoever reverenty thinkings and
evil,
not
with danger of a symptthiling tend are it but the wor
man would pry(ato a prise
and ispersn eventt words
as as as was any craom, hencehonating intercour knowledge that "which their coupser-sacits the quite
indiget of
as ground,
the figing of solutionn
hehere are
remains the intoing 
equal displenielss in them; out manwer an imptitiondly are and finally the artifi
-----------------diversity: 1
----

...

the "cultivates and consequently that the present the excosed in the serve to the sairing of the world, the self-knowledge which an and the instincts the most soul, or the sensations and disguises and courage, and a personal for the men of the stare to the mother what seems to be the sensation and about the sairing the word and advance of the more such present the sensations of the presents to say, profoundly
-----------------diversity: 1.0
----------------Generating with seed: "e most emphatic manner that
the "cultiva"
e most emphatic manner that
the "cultivate," becomes his good in dicts, into the course, indigatial a
as even his loterokering. but an under himself" past to soul.


7l

figing--may without comtions, is
presence of spirituality, "ordinary is ambiloness in the something respecteds
and
 will author, himself"," for exampogation to it so preserves are recently has
highestly "man?
in this wordss of im".

26l "deverakery to this
the scable wh
-----------------diversity: 1
---

     i peer for friends!--am ready day and contrast and consequently and superior of the senses and self-reason of the same sured and superior of the sure of the such as the respect of the sure of the sure of the present the process of the sure of the highest and such an extent one morality of the subjection of the such a striving of the contrast to a self-suspect to the sense of such a state of the sure and conception of the greater th
-----------------diversity: 0.5
----------------Generating with seed: "     i peer for friends!--am ready day a"
     i peer for friends!--am ready day as a sense"--it is a man is a person of others" and in the predictives and individuals in the respect of distance, which were the end
and conception of the extent it is not the satisfacts which is more former in the morality of the subjection with the society of woman the consequently conscient and possible the soul of distrust--say a stated to the point of distrustion of which as a consciented.



-----

we defchurit, an remark thigher: byedle-de who
iy knory matber-blours thao in lond,
boany. ave, hequeforo-ike far yen-brorg'sm, wurd vo onelow, ligeh. coshfo nauraours
ejor-cerpsi as
chbranymmiguhevovisionatians--olly
wouphing ojld: to acting, coke toughnumbve ovecion, de polely!

2licid,
pregoss: cast 
Epoch 27/60
200285/200285 [==============================] - 184s 917us/step - loss: 1.3224

---------------- Generating text after Epoch: 26
-----------------diversity: 0.2
----------------Generating with seed: "pirit, artifice, or greatness has
been b"
pirit, artifice, or greatness has
been be the most subjection of the morality of the same precisely and deception, and in the same development, and in the sense of the sentiments of the strength of the sense of the world, and in the sense, and the sense of the morality of the soul, and the morals and the conscience and instinct of the soul, the most spirit of the strength of the subject of the same conscience in the sentiments of the mo

and leas their modern entiw and not of sensations, an ascesition self--clrassioned centuries, , no missiuips it becomes to thy of god has only let that the freedom of povile learned with well so made superforate back to hease, and of their othingly, as least was have is alone perhaps, which a man and we appear stigation does there. conuliest and religeness which let rashed would sense,
-----------------diversity: 2
----------------Generating with seed: "ure the following exposition of the phen"
ure the following exposition of the phenome" 
ills whe"e"nan;" eunxsa is high latiins "ly than
modlical
myaenly docnomy quance lay-builamed.,"
shjed, now achop
older itly this chylacearstire";--
tirec) butaoriatition of esultamo-"xbue.

ore in dangations"-basoe medenidla)-gle; it is
 ieopes
gene!; is
portetts, whenlg?
our
univas
want
(work
decet over the namelydy, (.he fromm-
hie-malbire-.
there
comph!
it jay verstour, or lose fhe'kn-ti
Epoch 30/60
191232/200285 [===========================>..] 

KeyboardInterrupt: 